## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Import needed libraries for the entire lab

In [22]:
import pandas as pd 
from bs4 import BeautifulSoup 
import requests
import geocoder

#!conda install -c conda-forge geopy --ye
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Begining of first set of questions
    Collect postcodes from wikipedia with soup
    convert into a dataframe without an un assigned boroughs

In [2]:
wikiURL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rawPage = requests.get(wikiURL, 'html.parser')
soup = BeautifulSoup(rawPage.text, 'html.parser')
table = soup.find_all('table')[0]

Having collected the wiki page, build a data frame from it then remove the 'Not assigned' Boroughs

Having done that print the resulting shape of the dataFrame as requested

In [3]:
dataFrame = pd.read_html(str(table))[0]
dataFrame = dataFrame[dataFrame.Borough != 'Not assigned']
dataFrame.shape

(103, 3)

Begining of Second set of questions, collection of locations for the previous work

In [4]:
#collection from geocoder failed so csv used.
geoURL = 'https://cocl.us/Geospatial_data'
geoDF = pd.read_csv(geoURL)

#check the size of the geo dataframe
geoDF.shape

(103, 3)

In [6]:
#merge the data frames on post code
df_with_geo = dataFrame.merge(geoDF, left_on='Postal Code', right_on='Postal Code')

In [41]:
#check the results
df_with_geo.columns = df_with_geo.columns.str.replace(' ', '')
df_with_geo.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


setup foursquare info, ID and secret should be blank in submitted version

In [18]:
CLIENT_ID = '#' # your Foursquare ID
CLIENT_SECRET = '#' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VZAQKGZTYRUBC4NMWEYP3XS0BQBUJVQ1XSCAMJDCRRF2B0P2
CLIENT_SECRET:CJALW0S25ZR4K0DS31SYMB2TU3Z3VS0V3N4OS4RZTKTQ50CJ


Search for for farmers market within 1KM of centers and build a dataframe that will be merged with the 

In [44]:
search_query = 'Farmers Market'
radius = 1000
postCodeList = []
jsonResult = []

#df_with_geo.iloc[0]
#lat = df_with_geo.iloc[0]['Latitude']
#long = df_with_geo.iloc[0]['Longitude']
#postcode =  df_with_geo.iloc[0]['Postal Code']
for row in df_with_geo.itertuples(index=True, name='Pandas'):
    lat = row.Latitude
    long = row.Longitude
    postCode = row.PostalCode
    postCodeList.append(postCode)
    searchURL = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
    result = requests.get(searchURL).json()
    jsonResult.append(result)

toAdd = list(zip(postCodeList, jsonResult))
df_temp = pd.DataFrame(toAdd, columns = ['PostalCode', 'jsonResult'])
df_Farmers = df_with_geo.merge(df_temp, left_on='PostalCode', right_on='PostalCode')
#searchURL

Having established the locations of the farmers markets and the neighbourhoods they are in we can count them and split them into two groups those with more than 4 farmers markets and those with less, those with less might be the best choice to locate a new one.

In [109]:
postCodesMany = []
postCodesFew = []

for row in df_temp.itertuples(index=True, name='Pandas'):
    venues = row.jsonResult['response']['venues']
    df_venues = json_normalize(venues)
    if len(df_venues.index) < 4 :
        postCodesFew.append(row.PostalCode)
    else :
        postCodesMany.append(row.PostalCode)


indexToDrop = df_Farmers[df_Farmers.PostalCode.isin(postCodesFew) ].index
postDropMany = df_Farmers.drop(indexToDrop) # 

indexToDrop = df_Farmers[df_Farmers.PostalCode.isin(postCodesMany) ].index
postDropFew = df_Farmers.drop(indexToDrop)

df_PossibleComp = df_temp.drop(indexToDrop)

for row in df_PossibleComp.itertuples(index=True, name='Pandas'):
    venues = row.jsonResult['response']['venues']
    df_venues = json_normalize(venues)
    df_venues.columns = df_venues.columns.str.replace('.', '_')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


So now we are ready to draw a map, 
Green postal codes with fewer than 4 farmers markets
Yellow postal codes with 4 or more farmers markets
Blue location of farmers markets located within 1 KM the center of a postal code with fewer than 4 farmers markets



In [108]:


venues_map = folium.Map(location=torontoLoc , zoom_start=11) # generate map centred around the Conrad Hotel

#Red Circles for center of post codes
#for row in df_with_geo.itertuples(index=True, name='Pandas'):
for row in postDropFew.itertuples(index=True, name='Pandas'):
    lat = row.Latitude
    long = row.Longitude
    popup = row.Neighbourhood
    folium.vector_layers.CircleMarker(
        [lat, long],
        radius=10,
        color='green',
        popup=popup,
        fill = True,
        fill_color = 'green',
        fill_opacity = 0.6
    ).add_to(venues_map)
       
    
for row in postDropMany.itertuples(index=True, name='Pandas'):
    lat = row.Latitude
    long = row.Longitude
    popup = row.Neighbourhood
    folium.vector_layers.CircleMarker(
        [lat, long],
        radius=10,
        color='yellow',
        popup=popup,
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.3
    ).add_to(venues_map)
    
for row in df_PossibleComp.itertuples(index=True, name='Pandas'):
    venues = row.jsonResult['response']['venues']
    df_venues = json_normalize(venues)
    df_venues.columns = df_venues.columns.str.replace('.', '_')
    if ('location_lat' in df_venues.columns ):
        for lat, long in zip (df_venues.location_lat, df_venues.location_lng)  :
            #popup = row.Neighbourhood
            folium.vector_layers.CircleMarker(
                [lat, long],
                radius=3,
                color='red',
                popup="foo",
                fill = True,
                fill_color = 'red',
                fill_opacity = 0.3
            ).add_to(venues_map)
    

venues_map

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
